In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [2]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "libchewing", 6

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'src/chewingutil.c'}
bug.buggy_lines={('src/chewingutil.c', 544), ('src/chewingutil.c', 545)}
bug.coverage_df=


1  \
file                                              function           line         
/home/workspace/src/char.c                        CompUint16         51    2745   
                                                                     53    2745   
                                                  TerminateChar      56    27     
                                                                     59    27     
                                                                     60    27     
...                                                                        ..     
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 174   135    
                                                                     175   135    
                                                                     176   135    
                                                                     177   135    
                                                                     180   0      

                                                                             7  \
file                                              function           line        
/home/workspace/src/char.c                        CompUint16         51    352   
                                                                     53    352   
                                                  TerminateChar      56    1     
                                                                     59    1     
                                                                     60    1     
...                                                                       ..     
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 174   5     
                                                                     175   5     
                                                                     176   5     
                                                                     177   5     
                                                                     180   0     

                                                                               2  \
file                                              function           line          
/home/workspace/src/char.c                        CompUint16         51    75512   
                                                                     53    75512   
                                                  TerminateChar      56    14      
                                                                     59    14      
                                                                     60    14      
...                                                                        ..      
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 174   70      
                                                                     175   70      
                                                                     176   70      
                                                                     177   70      
                                                                     180   0       

                                                                             3  \
file                                              function           line        
/home/workspace/src/char.c                        CompUint16         51    172   
                                                                     53    172   
                                                  TerminateChar      56    2     
                                                                     59    2     
                                                                     60    2     
...                                                                       ..     
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 174   10    
                                                                     175   10    
                  

bug.failing_tests={'1'}
bug.failing_tests_GT={'1'}
bug.functions={('/home/workspace/src/porting_layer/src/plat_path.c', 'get_search_path'), ('/home/workspace/src/hash.c', 'HashInsert'), ('/home/workspace/src/hash.c', 'InitHash'), ('/home/workspace/src/choice.c', 'ChoiceFirstAvail'), ('/home/workspace/src/chewingio.c', 'chewing_get_maxChiSymbolLen'), , ...}
bug.id='libchewing-6'
bug.num_test_cases=15
bug.passing_tests={'3', '4', '15', '13', '5', '2', '12', '11', '7', '9', '6', '10', '14', '8'}
bug.path_to_cov_df='/home/coinse/Workspace/defects4cpp/coverage/libchewing_6_buggy/coverage_df.pkl'
bug.path_to_patch='/home/coinse/Workspace/defects4cpp/defects4cpp/taxonomy/libchewing/patch/0006-buggy.patch'
bug.path_to_test_results='/home/coinse/Workspace/defects4cpp/coverage/libchewing_6_buggy/test_results.json'
bug.source_files=/home/workspace/src/chewingio.c, /home/workspace/src/hash.c, /home/workspace/src/common/chewing-utf8-util.c, /home/workspace/src/tree.c, /home/workspace/src/porting_la

# Get spectra

In [3]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                              function           line                                                                                 
/home/workspace/src/char.c                        CompUint16         51    (/home/workspace/src/char.c, CompUint16, 51)                                   
                                                                     53    (/home/workspace/src/char.c, CompUint16, 53)                                   
                                                  TerminateChar      56    (/home/workspace/src/char.c, TerminateChar, 56)                                
                                                                     59    (/home/workspace/src/char.c, TerminateChar, 59)                                
                                                                     60    (/home/workspace/src/char.c, TerminateChar, 60)                                
...                                                                                                                    ...                                
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 174   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 174)   
                                                                     175   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 175)   
                                                                     176   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 176)   
                                                                     177   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 177)   
                                                                     180   (/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files, 180)   

                                                                           e_p  \
file                                              function           line        
/home/workspace/src/char.c                        CompUint16         51    6     
                                                                     53    6     
                                                  TerminateChar      56    10    
                                                                     59    10    
                                                                     60    10    
...                                                                        ..    
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 174   11    
                                                                     175   11    
                                                                     176   11    
                                                                     177   11    
                                                                     180   1     

                                                                           e_f  \
file                                              function           line        
/home/workspace/src/char.c                        CompUint16         51    1     
                                                                     53    1     
                                                  TerminateChar      56    1     
                                                                     59    1     
                                                                     60    1     
...                                                                       ..     
/home/workspace/src/porting_layer/src/plat_path.c find_path_by_files 174   1     
                                                                     175   1     
                                                                     176   1     
                                                                     177   1     
                                                                     180   0    

# Compute suspiciousness scores

In [4]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [5]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/src/chewingutil.c', 'ReleaseChiSymbolBuf')
fault idx: ('/home/workspace/src/chewingutil.c', 'ReleaseChiSymbolBuf', 545)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf, 545)",8,1,6,0,0.466667,0.333333,1366.0,1366.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/chewingio.c, chewing_handle_PageUp, 971)",0,1,14,0,1.0,1.0,148.0,148.0
1,"(/home/workspace/src/chewingio.c, chewing_handle_Up, 687)",0,1,14,0,1.0,1.0,148.0,148.0
2,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check, 110)",0,1,14,0,1.0,1.0,148.0,148.0
3,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check, 109)",0,1,14,0,1.0,1.0,148.0,148.0
4,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check, 108)",0,1,14,0,1.0,1.0,148.0,148.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/src/chewingio.c, chewing_handle_PageUp, 971)",0,1,14,0,1.0,1.0,148.0,148.0
1,"(/home/workspace/src/chewingio.c, chewing_handle_Up, 687)",0,1,14,0,1.0,1.0,148.0,148.0
2,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check, 110)",0,1,14,0,1.0,1.0,148.0,148.0
3,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check, 109)",0,1,14,0,1.0,1.0,148.0,148.0
4,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check, 108)",0,1,14,0,1.0,1.0,148.0,148.0


# Method level SBFL

In [6]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,51,6,1,8,0,0.600000,0.377964,1266.0,1266.0,/home/workspace/src/char.c,CompUint16,"(/home/workspace/src/char.c, CompUint16)"
1,53,6,1,8,0,0.600000,0.377964,1266.0,1266.0,/home/workspace/src/char.c,CompUint16,"(/home/workspace/src/char.c, CompUint16)"
2,56,10,1,4,0,0.333333,0.301511,2023.0,2023.0,/home/workspace/src/char.c,TerminateChar,"(/home/workspace/src/char.c, TerminateChar)"
3,59,10,1,4,0,0.333333,0.301511,2023.0,2023.0,/home/workspace/src/char.c,TerminateChar,"(/home/workspace/src/char.c, TerminateChar)"
4,60,10,1,4,0,0.333333,0.301511,2023.0,2023.0,/home/workspace/src/char.c,TerminateChar,"(/home/workspace/src/char.c, TerminateChar)"
...,...,...,...,...,...,...,...,...,...,...,...,...
2592,174,11,1,3,0,0.266667,0.288675,2075.0,2075.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"
2593,175,11,1,3,0,0.266667,0.288675,2075.0,2075.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"
2594,176,11,1,3,0,0.266667,0.288675,2075.0,2075.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"
2595,177,11,1,3,0,0.266667,0.288675,2075.0,2075.0,/home/workspace/src/porting_layer/src/plat_path.c,find_path_by_files,"(/home/workspace/src/porting_layer/src/plat_path.c, find_path_by_files)"


## Max aggregation

In [7]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",561,8,1,6,0,0.466667,146.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/src/zuin.c, ZuinRemoveLast)",678,0,1,14,0,1.0,28.0
1,"(/home/workspace/src/zuin.c, EndKeyProcess)",119,0,1,14,0,1.0,28.0
2,"(/home/workspace/src/tree.c, internal_release_Phrase)",341,0,1,14,0,1.0,28.0
3,"(/home/workspace/src/tree.c, FindInterval)",414,0,1,14,0,1.0,28.0
4,"(/home/workspace/src/tree.c, CountMatchCnnct)",875,0,1,14,0,1.0,28.0
5,"(/home/workspace/src/tree.c, CheckChoose)",279,0,1,14,0,1.0,28.0
6,"(/home/workspace/src/mod_aux.c, chewing_zuin_String)",101,0,1,14,0,1.0,28.0
7,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check)",113,0,1,14,0,1.0,28.0
8,"(/home/workspace/src/mod_aux.c, chewing_interval_hasNext)",175,0,1,14,0,1.0,28.0
9,"(/home/workspace/src/mod_aux.c, chewing_interval_Get)",187,0,1,14,0,1.0,28.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",561,8,1,6,0,0.333333,146.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/src/zuin.c, ZuinRemoveLast)",678,0,1,14,0,1.0,28.0
1,"(/home/workspace/src/zuin.c, EndKeyProcess)",119,0,1,14,0,1.0,28.0
2,"(/home/workspace/src/tree.c, internal_release_Phrase)",341,0,1,14,0,1.0,28.0
3,"(/home/workspace/src/tree.c, FindInterval)",414,0,1,14,0,1.0,28.0
4,"(/home/workspace/src/tree.c, CountMatchCnnct)",875,0,1,14,0,1.0,28.0
5,"(/home/workspace/src/tree.c, CheckChoose)",279,0,1,14,0,1.0,28.0
6,"(/home/workspace/src/mod_aux.c, chewing_zuin_String)",101,0,1,14,0,1.0,28.0
7,"(/home/workspace/src/mod_aux.c, chewing_zuin_Check)",113,0,1,14,0,1.0,28.0
8,"(/home/workspace/src/mod_aux.c, chewing_interval_hasNext)",175,0,1,14,0,1.0,28.0
9,"(/home/workspace/src/mod_aux.c, chewing_interval_Get)",187,0,1,14,0,1.0,28.0


## Line2Method voting (weight: sbfl score / dense rank)

In [8]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",0.205926,159.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,561,8,1,6,0,0.466667,0.333333,1366.0,1366.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",9.0,0.051852
1,546,8,1,6,0,0.466667,0.333333,1366.0,1366.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",9.0,0.051852
2,545,8,1,6,0,0.466667,0.333333,1366.0,1366.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",9.0,0.051852
3,543,8,1,6,0,0.466667,0.333333,1366.0,1366.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",9.0,0.051852
4,537,8,1,6,0,0.466667,0.333333,1366.0,1366.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",9.0,0.051852
5,559,2,0,12,1,-0.133333,0.000000,2517.0,2597.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",15.0,-0.008889
6,557,2,0,12,1,-0.133333,0.000000,2517.0,2597.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",15.0,-0.008889
7,556,2,0,12,1,-0.133333,0.000000,2517.0,2597.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",15.0,-0.008889
8,555,2,0,12,1,-0.133333,0.000000,2517.0,2597.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",15.0,-0.008889
9,554,2,0,12,1,-0.133333,0.000000,2517.0,2597.0,/home/workspace/src/chewingutil.c,ReleaseChiSymbolBuf,"(/home/workspace/src/chewingutil.c, ReleaseChiSymbolBuf)",15.0,-0.008889


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,122,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
1,118,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
2,117,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
3,116,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
4,115,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
5,114,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
6,113,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
7,112,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
8,111,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",1.0,1.000000
9,129,1,1,13,0,0.933333,0.707107,496.0,496.0,/home/workspace/src/chewingio.c,chooseCandidate,"(/home/workspace/src/chewingio.c, chooseCandidate)",2.0,0.353553


Top 5:


,meth,vote,rank
0,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",11.242641,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,889,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",1.0,1.000000
1,888,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",1.0,1.000000
2,885,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",1.0,1.000000
3,884,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",1.0,1.000000
4,883,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",1.0,1.000000
5,882,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",1.0,1.000000
6,881,0,1,14,0,1.000000,1.000000,148.0,148.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",1.0,1.000000
7,895,1,1,13,0,0.933333,0.707107,496.0,496.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",2.0,0.353553
8,894,1,1,13,0,0.933333,0.707107,496.0,496.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",2.0,0.353553
9,892,1,1,13,0,0.933333,0.707107,496.0,496.0,/home/workspace/src/chewingio.c,chewing_handle_Tab,"(/home/workspace/src/chewingio.c, chewing_handle_Tab)",2.0,0.353553


# 감상

단순히 passing test에 의해서 한 번도 실행되지 않고 failing test에 의해서 실행되는 method가 너무 많다 (sbfl max agg top max rank: 28)